# Setup

## Import Modules

In [28]:
# import packages
import os
import pandas as pd
import pickle
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

# set overwrite to true
arcpy.env.overwriteOutput = True

# set data frame display options
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.options.display.float_format = '{:,.2f}'.format

# set workspace and sde connections
arcpy.env.workspace = "C:\GIS\PROJECT\DevelopmentPotential\DevelopmentPotential.gdb"
sdeTabular = "F:\\GIS\\GIS_DATA\\Tabular.sde"
sdeBase = "F:\\GIS\\GIS_DATA\\Vector.sde"

## Data Transfer Function

In [29]:
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
   
    # Use list comprehension to build a dictionary from a da SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList)}  
   
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))

# Extract

## Get a Copy of Parcel Master

In [30]:
# copy parcel master to local gdb
# parcel master
parcels = sdeBase + "\\sde.SDE.Parcels\\sde.SDE.Parcel_Master"
# copy
arcpy.FeatureClassToFeatureClass_conversion(parcels,
                                             "C:\GIS\PROJECT\DevelopmentPotential\DevelopmentPotential.gdb",
                                            "parcel_master")

<Result 'C:\\GIS\\PROJECT\\DevelopmentPotential\\DevelopmentPotential.gdb\\parcel_master'>

## Get LTInfo Data

In [31]:
# create dataframes from https://qa.laketahoeinfo.org/WebServices/List
dfLTAPN    = pd.read_json("https://laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfIPES     = pd.read_json("https://laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfLCV      = pd.read_json("https://laketahoeinfo.org/WebServices/GetParcelsByLandCapability/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfDevBank  = pd.read_json("https://laketahoeinfo.org/WebServices/GetBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

## Add new fields to Parcel Master

In [32]:
# layer to add fields too
parcelMaster = 'parcel_master'

# Set local variables for field names
fieldName1 = "Vacant_Buildable"
fieldName2 = "MultiFamily_Allowed"
fieldName3 = "MultiFamily_UnitsPerAcre_Allowed"
fieldName4 = "SingleFamily_Allowed"
fieldName5 = "Commercial_Allowed"
fieldName6 = "Tourist_Allowed"
fieldName7 = "Existing_Residential_Units"
fieldName8 = "Existing_Tourist_Units"
fieldName9 = "Existing_CommercialFloorArea"
fieldName10 = "Max_Residential_Units"
fieldName11 = "Potential_Residential_Units"
fieldName12 = "TAZ"

# Execute AddField twice for two new fields
arcpy.AddField_management(parcelMaster, fieldName1, "TEXT")
arcpy.AddField_management(parcelMaster, fieldName2, "TEXT")
arcpy.AddField_management(parcelMaster, fieldName3, "SHORT")
arcpy.AddField_management(parcelMaster, fieldName4, "TEXT")
arcpy.AddField_management(parcelMaster, fieldName5, "TEXT")
arcpy.AddField_management(parcelMaster, fieldName6, "TEXT")
arcpy.AddField_management(parcelMaster, fieldName7, "SHORT")
arcpy.AddField_management(parcelMaster, fieldName8, "SHORT")
arcpy.AddField_management(parcelMaster, fieldName9, "DOUBLE")
arcpy.AddField_management(parcelMaster, fieldName10, "SHORT")
arcpy.AddField_management(parcelMaster, fieldName11, "SHORT")
arcpy.AddField_management(parcelMaster, fieldName12, "SHORT")

<Result 'C:\\GIS\\PROJECT\\DevelopmentPotential\\DevelopmentPotential.gdb\\parcel_master'>

## Inspect Data and Workspace

In [33]:
fields = arcpy.ListFields(parcelMaster)
for field in fields:
    print(field.name)

OBJECTID
Shape
APN
PPNO
HSE_NUMBR
UNIT_NUMBR
STR_DIR
STR_NAME
STR_SUFFIX
APO_ADDRESS
PSTL_TOWN
PSTL_STATE
PSTL_ZIP5
OWN_FIRST
OWN_LAST
OWN_FULL
MAIL_ADD1
MAIL_ADD2
MAIL_CITY
MAIL_STATE
MAIL_ZIP5
JURISDICTION
COUNTY
OWNERSHIP_TYPE
COUNTY_LANDUSE_CODE
COUNTY_LANDUSE_DESCRIPTION
TRPA_LANDUSE_DESCRIPTION
REGIONAL_LANDUSE
UNITS
BEDROOMS
BATHROOMS
ALLOWABLE_COVERAGE_BAILEY_SQFT
IMPERVIOUS_SURFACE_SQFT
SOIL_1974
SOIL_2003
HRA_NAME
WATERSHED_NUMBER
WATERSHED_NAME
PRIORITY_WATERSHED
FIREPD
WITHIN_TRPA_BNDY
LITTORAL
AS_LANDVALUE
AS_IMPROVALUE
AS_SUM
TAX_LANDVALUE
TAX_IMPROVALUE
TAX_SUM
TAX_YEAR
PLAN_ID
PLAN_NAME
ZONING_ID
ZONING_DESCRIPTION
TOWN_CENTER
LOCATION_TO_TOWNCENTER
INDEX_1987
LOCAL_PLAN_HYPERLINK
DESIGN_GUIDELINES_HYPERLINK
INDEX_1987_HYPERLINK
PARCEL_ACRES
PARCEL_SQFT
Shape_Length
Shape_Area
Vacant_Buildable
MultiFamily_Allowed
MultiFamily_UnitsPerAcre_Allowed
SingleFamily_Allowed
Commercial_Allowed
Tourist_Allowed
Existing_Residential_Units
Existing_Tourist_Units
Existing_CommercialF

In [34]:
arcpy.ListFeatureClasses()

['MF_Zoning',
 'dev_rights_2018',
 'SF_Allowed',
 'CFA_Allowed',
 'TouristAccommodation_Allowed',
 'ParcelDevelopment',
 'parcel_master']

In [35]:
fields = arcpy.ListFields('parcel_master')
for field in fields:
    print(field.name)

OBJECTID
Shape
APN
PPNO
HSE_NUMBR
UNIT_NUMBR
STR_DIR
STR_NAME
STR_SUFFIX
APO_ADDRESS
PSTL_TOWN
PSTL_STATE
PSTL_ZIP5
OWN_FIRST
OWN_LAST
OWN_FULL
MAIL_ADD1
MAIL_ADD2
MAIL_CITY
MAIL_STATE
MAIL_ZIP5
JURISDICTION
COUNTY
OWNERSHIP_TYPE
COUNTY_LANDUSE_CODE
COUNTY_LANDUSE_DESCRIPTION
TRPA_LANDUSE_DESCRIPTION
REGIONAL_LANDUSE
UNITS
BEDROOMS
BATHROOMS
ALLOWABLE_COVERAGE_BAILEY_SQFT
IMPERVIOUS_SURFACE_SQFT
SOIL_1974
SOIL_2003
HRA_NAME
WATERSHED_NUMBER
WATERSHED_NAME
PRIORITY_WATERSHED
FIREPD
WITHIN_TRPA_BNDY
LITTORAL
AS_LANDVALUE
AS_IMPROVALUE
AS_SUM
TAX_LANDVALUE
TAX_IMPROVALUE
TAX_SUM
TAX_YEAR
PLAN_ID
PLAN_NAME
ZONING_ID
ZONING_DESCRIPTION
TOWN_CENTER
LOCATION_TO_TOWNCENTER
INDEX_1987
LOCAL_PLAN_HYPERLINK
DESIGN_GUIDELINES_HYPERLINK
INDEX_1987_HYPERLINK
PARCEL_ACRES
PARCEL_SQFT
Shape_Length
Shape_Area
Vacant_Buildable
MultiFamily_Allowed
MultiFamily_UnitsPerAcre_Allowed
SingleFamily_Allowed
Commercial_Allowed
Tourist_Allowed
Existing_Residential_Units
Existing_Tourist_Units
Existing_CommercialF

In [36]:
fields = arcpy.ListFields('dev_rights_2018')
for field in fields:
    print(field.name)

OBJECTID
Shape
APN
res_final
tau_final
cfa_final
Shape_Length
Shape_Area


# Transform

## Add Commodities to Parcel Data

In [37]:
# update commodities data in new parcel feature class
fieldJoinCalc(parcelMaster, ['APN', 'Existing_Residential_Units'], 'dev_rights_2018', ['APN', 'res_final'])
print ("The 'Existing_Residential_Units' field in the parcel data has been updated")

# update commodities data in new parcel feature class
fieldJoinCalc(parcelMaster, ['APN', 'Existing_Tourist_Units'], 'dev_rights_2018', ['APN', 'tau_final'])
print ("The 'Existing_Tourist_Units' field in the parcel data has been updated")

# update commodities data in new parcel feature class
fieldJoinCalc(parcelMaster, ['APN', 'Existing_CommercialFloorArea'], 'dev_rights_2018', ['APN', 'cfa_final'])
print ("The 'Existing_CommercialFloorArea' field in the parcel data has been updated")

Started data transfer: 2020-02-23 14:41:33
Finished data transfer: 2020-02-23 14:41:36
The 'Existing_Residential_Units' field in the parcel data has been updated
Started data transfer: 2020-02-23 14:41:36
Finished data transfer: 2020-02-23 14:41:38
The 'Existing_Tourist_Units' field in the parcel data has been updated
Started data transfer: 2020-02-23 14:41:38
Finished data transfer: 2020-02-23 14:41:41
The 'Existing_CommercialFloorArea' field in the parcel data has been updated


## Add Zoning Information to Parcel Data

In [38]:
# spatial join for multifamily density
out_parcels = os.path.join("in_memory", "parcel_mf_density")
arcpy.SpatialJoin_analysis(parcelMaster, 'MF_Zoning', out_parcels, '#', '#', '#', 'HAVE_THEIR_CENTER_IN')
# transfer attributes to parcel layer
fieldJoinCalc('parcel_master', ['APN', 'MultiFamily_UnitsPerAcre_Allowed'], out_parcels, ['APN', 'Density'])
print ("The 'MF_Density_UnitsPerAcre' field in the parcel layer has been updated")

Started data transfer: 2020-02-23 14:41:52
Finished data transfer: 2020-02-23 14:41:55
The 'MF_Density_UnitsPerAcre' field in the parcel layer has been updated


In [39]:
# Add a selection to the layer based on location to features in another feature 
# class 
selectSF = arcpy.SelectLayerByLocation_management("parcel_master", "have_their_center_in", "SF_Allowed")

updateFieldsList = ['SingleFamily_Allowed']
with arcpy.da.UpdateCursor(selectSF, updateFieldsList) as cursor:  
    for row in cursor:  
        row[0] = "Yes"
        cursor.updateRow(row)

In [40]:
# Add a selection to the layer based on location to features in another feature 
# class 
selectMF = arcpy.SelectLayerByLocation_management("parcel_master", "have_their_center_in", "MF_Zoning")

updateFieldsList = ['MultiFamily_Allowed']
with arcpy.da.UpdateCursor(selectMF, updateFieldsList) as cursor:  
    for row in cursor:  
        row[0] = "Yes"
        cursor.updateRow(row)

In [41]:
# Add a selection to the layer based on location to features in another feature 
# class 
selectCFA = arcpy.SelectLayerByLocation_management("parcel_master", "have_their_center_in", "CFA_Allowed")

updateFieldsList = ['Commercial_Allowed']
with arcpy.da.UpdateCursor(selectCFA, updateFieldsList) as cursor:  
    for row in cursor:  
        row[0] = "Yes"
        cursor.updateRow(row)

In [42]:
# Add a selection to the layer based on location to features in another feature 
# class 
selectTAU = arcpy.SelectLayerByLocation_management("parcel_master", "have_their_center_in", "TouristAccommodation_Allowed")

updateFieldsList = ['Tourist_Allowed']
with arcpy.da.UpdateCursor(selectCFA, updateFieldsList) as cursor:  
    for row in cursor:  
        row[0] = "Yes"
        cursor.updateRow(row)

In [43]:
updateFieldsList = ['SingleFamily_Allowed', 'MultiFamily_Allowed', 'Max_Residential_Units', 'PARCEL_ACRES', 'MultiFamily_UnitsPerAcre_Allowed']
with arcpy.da.UpdateCursor(parcelMaster, updateFieldsList) as cursor:  
    for row in cursor:
        if row[1] == "Yes" and row[4] != None:
            row[2] = row[3] * row[4]
        elif row[0] == "Yes" and row[1] == None:
            row[2] = 1
        else:
            row[2] == None
        cursor.updateRow(row)

In [46]:
def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor if row[0] is not None})

myValues = unique_values('parcel_master' , 'TAZ')

print (myValues)

[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 2

## Add TAZ ID to Parcel Data

In [45]:
# TAZ feature class
taz = sdeBase + "\\sde.SDE.Transportation_Analysis_Zone"
# spatial join for multifamily density
out_parcels = os.path.join("in_memory", "parcel_taz")
# spatial join
arcpy.SpatialJoin_analysis('parcel_master', taz, out_parcels, '#', '#', '#', 'HAVE_THEIR_CENTER_IN')
# transfer attributes to parcel layer
fieldJoinCalc('parcel_master', ['APN', 'TAZ'], out_parcels, ['APN', 'TAZ_1'])
print ("The 'TAZ' field in the parcel layer has been updated")

Started data transfer: 2020-02-23 14:43:42
Finished data transfer: 2020-02-23 14:43:46
The 'TAZ' field in the parcel layer has been updated


## SDE Parcel_Master Data Frame

In [47]:
# create spatial dataframe from parcel master
sdfParcels = pd.DataFrame.spatial.from_featureclass('parcel_master')
sdfParcels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61263 entries, 0 to 61262
Data columns (total 73 columns):
OBJECTID                            61263 non-null int64
APN                                 61263 non-null object
PPNO                                61263 non-null float64
HSE_NUMBR                           61263 non-null int64
UNIT_NUMBR                          61263 non-null object
STR_DIR                             61263 non-null object
STR_NAME                            61263 non-null object
STR_SUFFIX                          61263 non-null object
APO_ADDRESS                         61263 non-null object
PSTL_TOWN                           61263 non-null object
PSTL_STATE                          61263 non-null object
PSTL_ZIP5                           61263 non-null object
OWN_FIRST                           61263 non-null object
OWN_LAST                            61263 non-null object
OWN_FULL                            61263 non-null object
MAIL_ADD1             

In [48]:
sdfParcels

,OBJECTID,APN,PPNO,HSE_NUMBR,UNIT_NUMBR,STR_DIR,STR_NAME,STR_SUFFIX,APO_ADDRESS,PSTL_TOWN,PSTL_STATE,PSTL_ZIP5,OWN_FIRST,OWN_LAST,OWN_FULL,MAIL_ADD1,MAIL_ADD2,MAIL_CITY,MAIL_STATE,MAIL_ZIP5,JURISDICTION,COUNTY,OWNERSHIP_TYPE,COUNTY_LANDUSE_CODE,COUNTY_LANDUSE_DESCRIPTION,TRPA_LANDUSE_DESCRIPTION,REGIONAL_LANDUSE,UNITS,BEDROOMS,BATHROOMS,ALLOWABLE_COVERAGE_BAILEY_SQFT,IMPERVIOUS_SURFACE_SQFT,SOIL_1974,SOIL_2003,HRA_NAME,WATERSHED_NUMBER,WATERSHED_NAME,PRIORITY_WATERSHED,FIREPD,WITHIN_TRPA_BNDY,LITTORAL,AS_LANDVALUE,AS_IMPROVALUE,AS_SUM,TAX_LANDVALUE,TAX_IMPROVALUE,TAX_SUM,TAX_YEAR,PLAN_ID,PLAN_NAME,ZONING_ID,ZONING_DESCRIPTION,TOWN_CENTER,LOCATION_TO_TOWNCENTER,INDEX_1987,LOCAL_PLAN_HYPERLINK,DESIGN_GUIDELINES_HYPERLINK,INDEX_1987_HYPERLINK,PARCEL_ACRES,PARCEL_SQFT,Vacant_Buildable,MultiFamily_Allowed,MultiFamily_UnitsPerAcre_Allowed,SingleFamily_Allowed,Commercial_Allowed,Tourist_Allowed,Existing_Residential_Units,Existing_Tourist_Units,Existing_CommercialFloorArea,Max_Residential_Units,Potential_Residential_Units,TAZ,SHAPE
0,1,048-042-03,"4,804,203.00",0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,NV,89704,,UNITED STATES OF AMERICA,UNITED STATES OF AMERICA,,,RENO,NV,00000,WA,WA,Federal,120,"Vacant, single family",Vacant,Conservation,0,0,0,"47,242.03",0.00,MsG,9405,Incline,18,THIRD CREEK,1,,1,0,261641,0,261641,747546,0,747546,2019,053,INCLINE LAKE,053,INCLINE LAKE,,Outside Buffer,,http://www.trpa.org/wp-content/uploads/053.pdf,N/A,,108.95,"4,745,879.49",None,None,nan,None,None,None,nan,nan,nan,nan,None,269.00,"{'rings': [[[765625.7054114696, 4354631.387734..."
1,2,048-140-01,"4,814,001.00",0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,NV,89511,,UNITED STATES OF AMERICA,UNITED STATES OF AMERICA,324 25TH ST,C/O DIRECTOR OF LANDS & MINERALS-USDA FOREST SER,OGDEN,UT,84401,WA,WA,Federal,120,"Vacant, single family",Vacant,,0,0,0,0.00,0.00,,,,0,,,,0,0,14000,0,14000,40000,0,40000,2019,,,,,,,,,,,1.80,"78,267.47",None,None,nan,None,None,None,nan,nan,nan,nan,None,nan,"{'rings': [[[765719.1469960865, 4354915.431587..."
2,3,048-140-02,"4,814,002.00",0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,NV,89511,,UNITED STATES OF AMERICA,UNITED STATES OF AMERICA,,,RENO,NV,00000,WA,WA,Federal,160,"Splinter, unbuildable: small size or shape",Vacant,,0,0,0,0.00,0.00,,,,0,,,,0,0,28,0,28,80,0,80,2019,,,,,,,,,,,0.08,"3,338.13",None,None,nan,None,None,None,nan,nan,nan,nan,None,nan,"{'rings': [[[765649.5035075517, 4354813.753804..."
3,4,048-140-03,"4,814,003.00",0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,NV,89451,,INCLINE VILLAGE GENERAL IMPROVEMENT DIST,INCLINE VILLAGE GENERAL IMPROVEMENT DIST,893 SOUTHWOOD BLVD,,INCLINE VILLAGE,NV,89451,WA,WA,Local,120,"Vacant, single family",Vacant,Conservation,0,0,0,"2,141.35",0.00,MsE,9404,Incline,18,THIRD CREEK,1,,1,0,602455,0,602455,1721300,0,1721300,2019,053,INCLINE LAKE,053,INCLINE LAKE,,Outside Buffer,,http://www.trpa.org/wp-content/uploads/053.pdf,N/A,,5.00,"217,843.29",None,None,nan,None,None,None,nan,nan,nan,nan,None,285.00,"{'rings': [[[765528.4230274847, 4354647.265832..."
4,5,048-140-04,"4,814,004.00",0,,,MOUNT ROSE,HWY,0 MOUNT ROSE HWY,,NV,89451,,UNITED STATES OF AMERICA,UNITED STATES OF AMERICA,,,RENO,NV,00000,WA,WA,Federal,100,"Vacant, other or unknown",Vacant,Conservation,0,0,0,"250,371.76","17,877.22",Ev,9001,Incline,18,THIRD CREEK,1,,1,0,1559172,0,1559172,4454778,0,4454778,2019,053,INCLINE LAKE,053,INCLINE LAKE,,Outside Buffer,,http://www.trpa.org/wp-content/uploads/053.pdf,N/A,,645.70,"28,126,585.80",None,None,nan,None,None,None,nan,nan,nan,nan,None,286.00,"{'rings': [[[765065.9830036154, 4355188.277743..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61258,61259,037-020-014,"37,020,014.00",0,,,,,0,,CA,96155,FOREST SERVICE & OWNERSHIP UNVERIFIED,USA,USA FOREST SERVICE & OWNERSHIP UNVERI

In [49]:
# select fields
dfSDEParcel = sdfParcels[['APN', 
                        'JURISDICTION',
                        'APO_ADDRESS',
                        'OWNERSHIP_TYPE',
                        'COUNTY_LANDUSE_DESCRIPTION',
                        'TRPA_LANDUSE_DESCRIPTION',
                        'TAZ',
                        'ZONING_ID',
                        'ZONING_DESCRIPTION',
                        'TOWN_CENTER',                     
                        'LOCATION_TO_TOWNCENTER',
                        'ALLOWABLE_COVERAGE_BAILEY_SQFT',
                        'IMPERVIOUS_SURFACE_SQFT',
                        'UNITS',
                        'BEDROOMS',
                        'BATHROOMS',
                        'HRA_NAME',
                        'WATERSHED_NAME',
                        'FIREPD',
                        'WITHIN_TRPA_BNDY',
                        'LITTORAL',
                        'AS_LANDVALUE',
                        'AS_IMPROVALUE',
                        'AS_SUM',
                        'TAX_LANDVALUE',
                        'TAX_IMPROVALUE',
                        'TAX_SUM',
                        'TAX_YEAR',
                        'PARCEL_SQFT',
                        'PARCEL_ACRES',
                        'Vacant_Buildable',
                        'MultiFamily_Allowed',
                        'MultiFamily_UnitsPerAcre_Allowed',
                        'SingleFamily_Allowed',
                        'Commercial_Allowed',
                        'Tourist_Allowed',
                        'Existing_Residential_Units',
                        'Existing_Tourist_Units',
                        'Existing_CommercialFloorArea',
                        'Max_Residential_Units',
                        'Potential_Residential_Units',
                        'SHAPE']]

sdfParcels_merge = dfSDEParcel
sdfParcels_merge

,APN,JURISDICTION,APO_ADDRESS,OWNERSHIP_TYPE,COUNTY_LANDUSE_DESCRIPTION,TRPA_LANDUSE_DESCRIPTION,TAZ,ZONING_ID,ZONING_DESCRIPTION,TOWN_CENTER,LOCATION_TO_TOWNCENTER,ALLOWABLE_COVERAGE_BAILEY_SQFT,IMPERVIOUS_SURFACE_SQFT,UNITS,BEDROOMS,BATHROOMS,HRA_NAME,WATERSHED_NAME,FIREPD,WITHIN_TRPA_BNDY,LITTORAL,AS_LANDVALUE,AS_IMPROVALUE,AS_SUM,TAX_LANDVALUE,TAX_IMPROVALUE,TAX_SUM,TAX_YEAR,PARCEL_SQFT,PARCEL_ACRES,Vacant_Buildable,MultiFamily_Allowed,MultiFamily_UnitsPerAcre_Allowed,SingleFamily_Allowed,Commercial_Allowed,Tourist_Allowed,Existing_Residential_Units,Existing_Tourist_Units,Existing_CommercialFloorArea,Max_Residential_Units,Potential_Residential_Units,SHAPE
0,048-042-03,WA,0 MOUNT ROSE HWY,Federal,"Vacant, single family",Vacant,269.00,053,INCLINE LAKE,,Outside Buffer,"47,242.03",0.00,0,0,0,Incline,THIRD CREEK,,1,0,261641,0,261641,747546,0,747546,2019,"4,745,879.49",108.95,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[765625.7054114696, 4354631.387734..."
1,048-140-01,WA,0 MOUNT ROSE HWY,Federal,"Vacant, single family",Vacant,nan,,,,,0.00,0.00,0,0,0,,,,0,0,14000,0,14000,40000,0,40000,2019,"78,267.47",1.80,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[765719.1469960865, 4354915.431587..."
2,048-140-02,WA,0 MOUNT ROSE HWY,Federal,"Splinter, unbuildable: small size or shape",Vacant,nan,,,,,0.00,0.00,0,0,0,,,,0,0,28,0,28,80,0,80,2019,"3,338.13",0.08,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[765649.5035075517, 4354813.753804..."
3,048-140-03,WA,0 MOUNT ROSE HWY,Local,"Vacant, single family",Vacant,285.00,053,INCLINE LAKE,,Outside Buffer,"2,141.35",0.00,0,0,0,Incline,THIRD CREEK,,1,0,602455,0,602455,1721300,0,1721300,2019,"217,843.29",5.00,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[765528.4230274847, 4354647.265832..."
4,048-140-04,WA,0 MOUNT ROSE HWY,Federal,"Vacant, other or unknown",Vacant,286.00,053,INCLINE LAKE,,Outside Buffer,"250,371.76","17,877.22",0,0,0,Incline,THIRD CREEK,,1,0,1559172,0,1559172,4454778,0,4454778,2019,"28,126,585.80",645.70,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[765065.9830036154, 4355188.277743..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61258,037-020-014,EL,0,Federal,VAC RES RURAL RES. LAND 20+ MINOR NON-RES IMPR,Vacant,nan,,,,,"2,886.07",0.00,0,0,,,,LAKE VALLEY FPD,0,0,0,0,0,0,0,0,2019,"7,319,504.76",168.03,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[755981.943499105, 4297124.9795018..."
61259,038-010-015,EL,0,Federal,VAC RES RURAL RES. LAND 20+ MINOR NON-RES IMPR,Vacant,nan,,,,,54.77,0.00,0,0,,,,,0,0,0,0,0,0,0,0,2019,"7,030,322.67",161.39,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[751787.4702896336, 4301170.615835..."
61260,038-030-046,EL,0,Federal,VAC RES RURAL RES. LAND 20+ MINOR NON-RES IMPR,Vacant,nan,,,,,"15,442.10",0.00,0,0,,,,,0,0,0,0,0,0,0,0,2019,"3,603,840.26",82.73,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[756875.7088519661, 4295072.845739..."
61261,014-080-004,EL,0,Federal,RLU MSC ENV. SENSITIVE LAND - RESTRICTED USE,Open Space,111.00,,,,,"97,817.78",0.00,0,0,,,,,0,0,0,0,0,0,0,0,2019,"29,176,677.60",669.80,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[747793.03904723, 4305675.45699422..."


## Parcel Tracker Data Frame

In [50]:
dfLTAPN.head()

,APN,ParcelNickname,ParcelSize,Status,RetiredFromDevelopment,IsAutoImported,OwnerName,ParcelAddress,Jurisdiction,ParcelNotes,LocalPlan,FireDistrict,ParcelWatershed,BMPStatus,HRA,HasMooringRegistration,SFRUU,RBU,TAU,CFA,RFA,TFA,PRUU,MFRUU
0,001-010-02,None,nan,Inactive,No,Yes,None,Address not available,Unknown or unspecified,None,None,None,None,Not Certified,None,False,0,0,0,0,0,0,0,0
1,001-010-03,None,nan,Inactive,No,Yes,None,Address not available,Unknown or unspecified,None,None,None,None,Not Certified,None,False,0,0,0,0,0,0,0,0
2,001-010-04,None,nan,Inactive,No,Yes,None,Address not available,Unknown or unspecified,None,None,None,None,Not Certified,None,False,0,0,0,0,0,0,0,0
3,001-010-05,None,nan,Inactive,No,Yes,None,Address not available,Unknown or unspecified,None,None,None,None,Not Certified,None,False,0,0,0,0,0,0,0,0
4,001-010-07,None,nan,Inactive,No,Yes,None,Address not available,Unknown or unspecified,None,None,None,None,Not Certified,None,False,0,0,0,0,0,0,0,0


In [51]:
dfLTAPN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70960 entries, 0 to 70959
Data columns (total 24 columns):
APN                       70960 non-null object
ParcelNickname            206 non-null object
ParcelSize                63483 non-null float64
Status                    70960 non-null object
RetiredFromDevelopment    70960 non-null object
IsAutoImported            70960 non-null object
OwnerName                 63255 non-null object
ParcelAddress             70960 non-null object
Jurisdiction              70960 non-null object
ParcelNotes               5513 non-null object
LocalPlan                 61097 non-null object
FireDistrict              62860 non-null object
ParcelWatershed           60317 non-null object
BMPStatus                 70960 non-null object
HRA                       62278 non-null object
HasMooringRegistration    70960 non-null bool
SFRUU                     70960 non-null int64
RBU                       70960 non-null int64
TAU                       70960 n

In [52]:
# select fields
dfLTinfoParcel = dfLTAPN[['APN', 
                          'RBU',
                          'RFA',
                          'SFRUU',
                          'CFA',
                          'TAU',
                          'TFA',
                          'BMPStatus',
                          'RetiredFromDevelopment',
                          'Status',
                          'ParcelSize']]

dfLTInfo_merge = dfLTinfoParcel[(dfLTinfoParcel.Status == "Active") & (dfLTinfoParcel.RetiredFromDevelopment == "No")]

dfLTInfo_merge

,APN,RBU,RFA,SFRUU,CFA,TAU,TFA,BMPStatus,RetiredFromDevelopment,Status,ParcelSize
229,001-161-06,0,0,0,0,0,0,Not Certified,No,Active,nan
1243,007-011-10,0,0,0,0,0,0,Not Certified,No,Active,"1,731,864.00"
1244,007-011-12,0,0,0,0,0,0,Not Certified,No,Active,"5,222,194.00"
1247,007-011-15,0,0,0,0,0,0,Not Certified,No,Active,"8,777,558.00"
1248,007-011-16,0,0,0,0,0,0,Not Certified,No,Active,"8,767,857.00"
...,...,...,...,...,...,...,...,...,...,...,...
69500,881-091-030,0,0,0,0,0,0,Not Certified,No,Active,"2,789,082.00"
69501,881-091-031,0,0,0,0,0,0,Not Certified,No,Active,"1,548,670.00"
69504,881-093-019,0,0,0,0,0,0,Not Certified,No,Active,"1,612,754.00"
69506,881-278-093,0,0,0,0,0,0,Not Certified,No,Active,"14,040.00"


## LCV Data Frame

In [55]:
dfLCV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21539 entries, 0 to 21538
Data columns (total 30 columns):
APN                    21539 non-null object
Status                 21539 non-null object
ParcelNickname         140 non-null object
TotalAreaSqFt          20959 non-null float64
UpdatedBy              21539 non-null object
UpdatedOn              21539 non-null object
DeterminationDate      21539 non-null object
EstimatedOrVerified    21539 non-null object
SitePlanUrl            21539 non-null object
AccelaCAPRecord        5540 non-null object
Bailey1aPresent        21539 non-null object
Bailey1aSqFt           503 non-null float64
Bailey1bPresent        21539 non-null object
Bailey1bSqFt           959 non-null float64
Bailey1cPresent        21539 non-null object
Bailey1cSqFt           51 non-null float64
Bailey2Present         21539 non-null object
Bailey2SqFt            258 non-null float64
Bailey3Present         21539 non-null object
Bailey3SqFt            275 non-null float64

In [56]:
dfLCV.head()

,APN,Status,ParcelNickname,TotalAreaSqFt,UpdatedBy,UpdatedOn,DeterminationDate,EstimatedOrVerified,SitePlanUrl,AccelaCAPRecord,Bailey1aPresent,Bailey1aSqFt,Bailey1bPresent,Bailey1bSqFt,Bailey1cPresent,Bailey1cSqFt,Bailey2Present,Bailey2SqFt,Bailey3Present,Bailey3SqFt,Bailey4Present,Bailey4SqFt,Bailey5Present,Bailey5SqFt,Bailey6Present,Bailey6SqFt,Bailey7Present,Bailey7SqFt,IPESPresent,IPESSqFt
0,001-050-09,Inactive,None,nan,Adele Gerz,08/14/2018 4:33 PM,08/14/2018 12:00 AM,Estimated,None,None,No,nan,No,nan,No,nan,No,nan,No,nan,No,nan,No,nan,No,nan,No,nan,No,nan
1,005-113-04,Inactive,None,nan,Ken Kasman,06/02/2016 11:16 AM,06/02/2016 12:00 AM,Verified,https://laketahoeinfo.org/FileResource/Display...,20020489STD,No,nan,No,nan,No,nan,Yes,nan,No,nan,Yes,nan,No,nan,No,nan,No,nan,No,nan
2,005-113-05,Inactive,None,nan,Ken Kasman,06/02/2016 10:57 AM,10/04/2001 12:00 AM,Verified,https://laketahoeinfo.org/FileResource/Display...,20020488STD,No,nan,No,nan,No,nan,Yes,nan,No,nan,Yes,nan,No,nan,No,nan,No,nan,No,nan
3,005-331-08,Inactive,None,nan,Jeanne McNamara,12/01/2017 11:24 AM,08/01/2001 12:00 AM,Verified,None,None,Yes,nan,No,nan,No,nan,Yes,nan,No,nan,Yes,nan,No,nan,No,nan,No,nan,No,nan
4,007-011-06,Active,None,"854,724.00",Ken Kasman,12/19/2016 4:38 PM,12/19/2016 4:38 PM,Verified,None,None,No,nan,No,nan,No,nan,No,nan,No,nan,No,nan,No,nan,No,nan,No,nan,Yes,"522,700.00"


In [57]:
# select fields
dfCoverageAllowed = dfLCV[['APN','Bailey1aSqFt', 'Bailey1bSqFt', 'Bailey1cSqFt','Bailey2SqFt','Bailey3SqFt','Bailey4SqFt', 'Bailey5SqFt', 'Bailey6SqFt', 'Bailey7SqFt','TotalAreaSqFt', 'Status', 'EstimatedOrVerified']]

In [58]:
# add coverage allowed field
dfCoverageAllowed["Developable_Area"] = np.nan

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [59]:
# calculate coverage allowed
dfCoverageAllowed['Developable_Area'] = dfCoverageAllowed[dfCoverageAllowed.columns[:-4]].sum(axis=1)
dfCoverageAllowed

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,APN,Bailey1aSqFt,Bailey1bSqFt,Bailey1cSqFt,Bailey2SqFt,Bailey3SqFt,Bailey4SqFt,Bailey5SqFt,Bailey6SqFt,Bailey7SqFt,TotalAreaSqFt,Status,EstimatedOrVerified,Developable_Area
0,001-050-09,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Inactive,Estimated,0.00
1,005-113-04,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Inactive,Verified,0.00
2,005-113-05,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Inactive,Verified,0.00
3,005-331-08,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Inactive,Verified,0.00
4,007-011-06,nan,nan,nan,nan,nan,nan,nan,nan,nan,"854,724.00",Active,Verified,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21534,1418-34-610-010,nan,nan,nan,nan,nan,nan,nan,nan,nan,"15,160.00",Active,Verified,0.00
21535,1418-34-610-012,nan,nan,nan,nan,nan,"15,919.00",nan,"12,397.00",nan,"28,367.00",Active,Verified,"28,316.00"
21536,1418-34-610-015,nan,nan,nan,"13,991.00",nan,nan,nan,nan,nan,"11,453.00",Active,Verified,"13,991.00"
21537,1418-34-610-016,nan,nan,nan,nan,nan,"6,152.00",nan,"9,976.00",nan,"16,089.00",Active,Verified,"16,128.00"


In [60]:
LCVquery = dfCoverageAllowed[(dfCoverageAllowed.Status == "Active") 
                            & (dfCoverageAllowed.EstimatedOrVerified == "Verified") 
                            & (dfCoverageAllowed.Developable_Area > 0)]

dfLCV_merge = LCVquery[['APN', 
                      'Developable_Area',
                      'TotalAreaSqFt', 
                      'Status', 
                      'EstimatedOrVerified']]
dfLCV_merge

,APN,Developable_Area,TotalAreaSqFt,Status,EstimatedOrVerified
5,007-011-23,"86,112.00","85,915.00",Active,Verified
7,014-021-011,"90,206.00","153,288.00",Active,Verified
19,014-233-005,"9,952.00","10,001.00",Active,Verified
27,014-236-004,"4,477.00","10,014.00",Active,Verified
33,014-236-012,"10,000.00","10,013.00",Active,Verified
...,...,...,...,...,...
21531,1418-34-610-004,"17,596.00","17,591.00",Active,Verified
21533,1418-34-610-007,"23,179.00","23,334.00",Active,Verified
21535,1418-34-610-012,"28,316.00","28,367.00",Active,Verified
21536,1418-34-610-015,"13,991.00","11,453.00",Active,Verified


## IPES Data Frame

In [61]:
dfIPES.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14643 entries, 0 to 14642
Data columns (total 27 columns):
ScoreSheetUrl                     14643 non-null object
APN                               14643 non-null object
Status                            14643 non-null object
ParcelNickname                    14643 non-null object
IPESScore                         14641 non-null float64
IPESScoreType                     14643 non-null object
BaseAllowableCoveragePercent      14643 non-null object
IPESTotalAllowableCoverageSqFt    843 non-null float64
ParcelHasDOAC                     14643 non-null object
HistoricOrImportedIpesScore       14643 non-null object
CalculationDate                   14643 non-null object
FieldEvaluationDate               14643 non-null object
RelativeErosionHazardScore        14461 non-null float64
RunoffPotentialScore              14461 non-null float64
AccessScore                       14461 non-null float64
UtilityInSEZScore                 14461 non-null

In [62]:
dfIPES.head()

,ScoreSheetUrl,APN,Status,ParcelNickname,IPESScore,IPESScoreType,BaseAllowableCoveragePercent,IPESTotalAllowableCoverageSqFt,ParcelHasDOAC,HistoricOrImportedIpesScore,CalculationDate,FieldEvaluationDate,RelativeErosionHazardScore,RunoffPotentialScore,AccessScore,UtilityInSEZScore,ConditionOfWatershedScore,AbilityToRevegetateScore,WaterQualityImprovementsScore,ProximityToLakeScore,LimitedIncentivePoints,TotalParcelArea,IPESBuildingSiteArea,SEZLandArea,SEZSetbackArea,InternalNotes,PublicNotes
0,https://parcels.laketahoeinfo.org/IpesScore/De...,001-010-05,Inactive,,815.00,Official,0%,nan,Unknown,Yes,,,345.00,44.00,158.00,110.00,58.00,50.00,40.00,10.00,0.00,nan,"2,831,400.00","20,000.00","4,000.00",,
1,https://parcels.laketahoeinfo.org/IpesScore/De...,001-010-07,Inactive,,673.00,Information Only,15%,nan,Unknown,Yes,02/26/1993 12:00 AM,,286.00,98.00,50.00,110.00,58.00,21.00,40.00,10.00,0.00,nan,"653,400.00",0.00,0.00,,
2,https://parcels.laketahoeinfo.org/IpesScore/De...,001-010-07,Inactive,,718.00,Information Only,16%,nan,Unknown,Yes,03/20/2002 12:00 AM,,292.00,98.00,85.00,110.00,58.00,25.00,40.00,10.00,0.00,nan,"653,400.00",0.00,0.00,"Original M score, moved from appendix",
3,https://parcels.laketahoeinfo.org/IpesScore/De...,001-020-03,Inactive,,699.00,Official,17%,nan,Unknown,Yes,03/22/2002 12:00 AM,,227.00,167.00,104.00,110.00,44.00,17.00,20.00,10.00,0.00,nan,"1,715,800.00",0.00,0.00,Used to be a n score;removed from appendx; ear...,
4,https://parcels.laketahoeinfo.org/IpesScore/De...,001-020-03,Inactive,,650.00,Information Only,4%,nan,Unknown,Yes,12/31/1990 12:00 AM,,274.00,44.00,127.00,110.00,44.00,21.00,20.00,10.00,0.00,nan,"1,715,800.00",0.00,0.00,,


In [63]:
# select fields
dfIPESScore = dfIPES[['APN',
                      'BaseAllowableCoveragePercent', 
                      'IPESScore', 
                      'IPESScoreType',
                      'IPESTotalAllowableCoverageSqFt',
                      'TotalParcelArea', 
                      'Status']]

In [64]:
dfIPES_merge = dfIPESScore[(dfIPESScore.Status == "Active") 
                            & (dfIPESScore.IPESScoreType == "Official")]

dfIPES_merge

,APN,BaseAllowableCoveragePercent,IPESScore,IPESScoreType,IPESTotalAllowableCoverageSqFt,TotalParcelArea,Status
243,007-011-06,9%,476.00,Official,nan,nan,Active
390,014-021-011,11%,756.00,Official,"6,944.00",nan,Active
391,014-231-005,24%,826.00,Official,nan,nan,Active
392,014-231-007,24%,830.00,Official,nan,nan,Active
393,014-231-009,26%,874.00,Official,nan,nan,Active
...,...,...,...,...,...,...,...
14636,1418-34-610-001,28%,928.00,Official,nan,nan,Active
14637,1418-34-610-002,8%,772.00,Official,nan,nan,Active
14638,1418-34-610-005,24%,858.00,Official,nan,nan,Active
14639,1418-34-610-010,5%,727.00,Official,732.00,nan,Active


## Development Rights Dataframe

In [65]:
dfDevBank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498 entries, 0 to 1497
Data columns (total 11 columns):
APN                     1498 non-null object
DevelopmentRight        1498 non-null object
LandCapability          1498 non-null object
IPESScore               371 non-null float64
BankedQuanitity         1498 non-null int64
CurrentInventory        1498 non-null int64
Jurisdiction            1498 non-null object
LocalPlan               1478 non-null object
DateBankedOrApproved    1498 non-null object
HRA                     1496 non-null object
LastUpdated             1498 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 128.9+ KB


In [66]:
dfDevBank.head()

,APN,DevelopmentRight,LandCapability,IPESScore,BankedQuanitity,CurrentInventory,Jurisdiction,LocalPlan,DateBankedOrApproved,HRA,LastUpdated
0,014-303-002,Potential Residential Unit of Use (PRUU),IPES,733.00,1,1,"El Dorado County, CA (ELDO)",TAHOMA RESIDENTIAL,12/03/2018,McKinney Bay,12/03/2018
1,015-063-005,Coverage (potential),IPES,0.00,63,63,"El Dorado County, CA (ELDO)",TAHOMA RESIDENTIAL,07/09/2004,McKinney Bay,01/09/2018
2,015-103-008,Coverage (potential),IPES,0.00,63,63,"El Dorado County, CA (ELDO)",TAHOMA RESIDENTIAL,04/03/2006,McKinney Bay,10/13/2017
3,015-103-008,Potential Residential Unit of Use (PRUU),IPES,0.00,1,1,"El Dorado County, CA (ELDO)",TAHOMA RESIDENTIAL,04/03/2006,McKinney Bay,10/13/2017
4,015-113-003,Coverage (potential),Bailey 1b,nan,63,63,"El Dorado County, CA (ELDO)",TAHOMA RESIDENTIAL,09/03/2015,McKinney Bay,10/16/2017


In [67]:
# select fields
dfBank_merge = dfDevBank[['APN',
                    'DevelopmentRight',
                    'LandCapability',
                    'BankedQuanitity',
                    'CurrentInventory',
                    'DateBankedOrApproved',
                    'LastUpdated'
                      ]]
dfBank_merge

,APN,DevelopmentRight,LandCapability,BankedQuanitity,CurrentInventory,DateBankedOrApproved,LastUpdated
0,014-303-002,Potential Residential Unit of Use (PRUU),IPES,1,1,12/03/2018,12/03/2018
1,015-063-005,Coverage (potential),IPES,63,63,07/09/2004,01/09/2018
2,015-103-008,Coverage (potential),IPES,63,63,04/03/2006,10/13/2017
3,015-103-008,Potential Residential Unit of Use (PRUU),IPES,1,1,04/03/2006,10/13/2017
4,015-113-003,Coverage (potential),Bailey 1b,63,63,09/03/2015,10/16/2017
...,...,...,...,...,...,...,...
1493,1418-34-310-010,Coverage (potential),Bailey 4,1194,1194,12/28/2007,08/20/2016
1494,1418-34-401-007,Coverage (potential),IPES,1063,1063,06/09/2016,06/09/2016
1495,1418-34-401-010,Single-Family Residential Unit of Use (SFRUU),Bailey 4,1,1,06/10/2013,01/03/2016
1496,1418-34-610-012,Coverage (potential),Bailey 4,2014,2014,08/05/2016,09/07/2016


## Merge Data Frames

In [68]:
from functools import reduce
data_frames = [sdfParcels_merge, dfLTInfo_merge, dfLCV_merge, dfIPES_merge, dfBank_merge]

# merge dataframes
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['APN'],
                                            how='left'), data_frames)

In [69]:
df_merged

,APN,JURISDICTION,APO_ADDRESS,OWNERSHIP_TYPE,COUNTY_LANDUSE_DESCRIPTION,TRPA_LANDUSE_DESCRIPTION,TAZ,ZONING_ID,ZONING_DESCRIPTION,TOWN_CENTER,LOCATION_TO_TOWNCENTER,ALLOWABLE_COVERAGE_BAILEY_SQFT,IMPERVIOUS_SURFACE_SQFT,UNITS,BEDROOMS,BATHROOMS,HRA_NAME,WATERSHED_NAME,FIREPD,WITHIN_TRPA_BNDY,LITTORAL,AS_LANDVALUE,AS_IMPROVALUE,AS_SUM,TAX_LANDVALUE,TAX_IMPROVALUE,TAX_SUM,TAX_YEAR,PARCEL_SQFT,PARCEL_ACRES,Vacant_Buildable,MultiFamily_Allowed,MultiFamily_UnitsPerAcre_Allowed,SingleFamily_Allowed,Commercial_Allowed,Tourist_Allowed,Existing_Residential_Units,Existing_Tourist_Units,Existing_CommercialFloorArea,Max_Residential_Units,Potential_Residential_Units,SHAPE,RBU,RFA,SFRUU,CFA,TAU,TFA,BMPStatus,RetiredFromDevelopment,Status_x,ParcelSize,Developable_Area,TotalAreaSqFt,Status_y,EstimatedOrVerified,BaseAllowableCoveragePercent,IPESScore,IPESScoreType,IPESTotalAllowableCoverageSqFt,TotalParcelArea,Status,DevelopmentRight,LandCapability,BankedQuanitity,CurrentInventory,DateBankedOrApproved,LastUpdated
0,048-042-03,WA,0 MOUNT ROSE HWY,Federal,"Vacant, single family",Vacant,269.00,053,INCLINE LAKE,,Outside Buffer,"47,242.03",0.00,0,0,0,Incline,THIRD CREEK,,1,0,261641,0,261641,747546,0,747546,2019,"4,745,879.49",108.95,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[765625.7054114696, 4354631.387734...",0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,Active,"4,717,362.00","4,693,108.00","4,745,879.00",Active,Verified,NaN,nan,NaN,nan,nan,NaN,NaN,NaN,nan,nan,NaN,NaN
1,048-140-01,WA,0 MOUNT ROSE HWY,Federal,"Vacant, single family",Vacant,nan,,,,,0.00,0.00,0,0,0,,,,0,0,14000,0,14000,40000,0,40000,2019,"78,267.47",1.80,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[765719.1469960865, 4354915.431587...",0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,Active,"78,267.00",nan,nan,NaN,NaN,NaN,nan,NaN,nan,nan,NaN,NaN,NaN,nan,nan,NaN,NaN
2,048-140-02,WA,0 MOUNT ROSE HWY,Federal,"Splinter, unbuildable: small size or shape",Vacant,nan,,,,,0.00,0.00,0,0,0,,,,0,0,28,0,28,80,0,80,2019,"3,338.13",0.08,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[765649.5035075517, 4354813.753804...",0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,Active,"3,338.00",nan,nan,NaN,NaN,NaN,nan,NaN,nan,nan,NaN,NaN,NaN,nan,nan,NaN,NaN
3,048-140-03,WA,0 MOUNT ROSE HWY,Local,"Vacant, single family",Vacant,285.00,053,INCLINE LAKE,,Outside Buffer,"2,141.35",0.00,0,0,0,Incline,THIRD CREEK,,1,0,602455,0,602455,1721300,0,1721300,2019,"217,843.29",5.00,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[765528.4230274847, 4354647.265832...",0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,Active,"217,800.00","205,797.00","217,843.00",Active,Verified,NaN,nan,NaN,nan,nan,NaN,NaN,NaN,nan,nan,NaN,NaN
4,048-140-04,WA,0 MOUNT ROSE HWY,Federal,"Vacant, other or unknown",Vacant,286.00,053,INCLINE LAKE,,Outside Buffer,"250,371.76","17,877.22",0,0,0,Incline,THIRD CREEK,,1,0,1559172,0,1559172,4454778,0,4454778,2019,"28,126,585.80",645.70,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[765065.9830036154, 4355188.277743...",0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,Active,"28,120,829.00","25,290,253.00","28,126,586.00",Active,Verified,NaN,nan,NaN,nan,nan,NaN,NaN,NaN,nan,nan,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61672,037-020-014,EL,0,Federal,VAC RES RURAL RES. LAND 20+ MINOR NON-RES IMPR,Vacant,nan,,,,,"2,886.07",0.00,0,0,,,,LAKE VALLEY FPD,0,0,0,0,0,0,0,0,2019,"7,319,504.76",168.03,None,None,nan,None,None,None,nan,nan,nan,nan,None,"{'rings': [[[755981.943499105, 4297124.9795018...",0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,Active,"7,319,505.00",nan,nan,NaN,NaN,NaN,nan,NaN,nan,nan,NaN,NaN,NaN,nan,nan,NaN,NaN
61673,038-010-015,EL,0,Federal,VAC RES RURAL RES. LAND 20+ MINOR 

In [70]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61677 entries, 0 to 61676
Data columns (total 68 columns):
APN                                 61677 non-null object
JURISDICTION                        61677 non-null object
APO_ADDRESS                         61677 non-null object
OWNERSHIP_TYPE                      61677 non-null object
COUNTY_LANDUSE_DESCRIPTION          61677 non-null object
TRPA_LANDUSE_DESCRIPTION            61677 non-null object
TAZ                                 61534 non-null float64
ZONING_ID                           61677 non-null object
ZONING_DESCRIPTION                  61677 non-null object
TOWN_CENTER                         61677 non-null object
LOCATION_TO_TOWNCENTER              61677 non-null object
ALLOWABLE_COVERAGE_BAILEY_SQFT      61670 non-null float64
IMPERVIOUS_SURFACE_SQFT             61677 non-null float64
UNITS                               61677 non-null object
BEDROOMS                            61677 non-null object
BATHROOMS         

In [71]:
df = df_merged

## Define Vacant Buildable

In [72]:
df.Vacant_Buildable = np.nan

In [75]:
df.loc[(((df.TRPA_LANDUSE_DESCRIPTION == "Vacant")
        & (df.OWNERSHIP_TYPE == "Private")
        & (df.RetiredFromDevelopment == 'No')
        & ((df.JURISDICTION.isin(['EL','CSLT','DG','WA','CC'])) & (df.IPESScore > 0))
        | (df.TRPA_LANDUSE_DESCRIPTION == "Vacant")
        & (df.OWNERSHIP_TYPE == "Private") 
        & (df.RetiredFromDevelopment == 'No')
        & ((df.JURISDICTION == "PL") & (df.IPESScore>726))), 'Vacant_Buildable')] = 'Yes'

In [76]:
df.loc[(((df.OWNERSHIP_TYPE == "Private")
        & (df.IMPERVIOUS_SURFACE_SQFT < 1000)
        & (df.RetiredFromDevelopment == 'No')
        & ((df.JURISDICTION.isin(['EL','CSLT','DG','WA','CC'])) & (df.IPESScore > 0))
        | (df.OWNERSHIP_TYPE == "Private") 
        & (df.RetiredFromDevelopment == 'No')
        & (df.IMPERVIOUS_SURFACE_SQFT < 1000)
        & ((df.JURISDICTION == "PL") & (df.IPESScore>726))), 'Vacant_Buildable')] = 'Maybe'

## Calculate Potential Units

In [77]:
df.Potential_Residential_Units = df.Max_Residential_Units - df.Existing_Residential_Units

## Calculate Potential Coverage

In [78]:
df["Potential_Coverage"] = df.ALLOWABLE_COVERAGE_BAILEY_SQFT - df.IMPERVIOUS_SURFACE_SQFT

## Calculate Dwelling Units Per Acre

In [79]:
df["DwellingUnits_PerAcre"] = df.Existing_Residential_Units / df.PARCEL_ACRES

# Load

## Data Frame to Feature Class

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61677 entries, 0 to 61676
Data columns (total 70 columns):
APN                                 61677 non-null object
JURISDICTION                        61677 non-null object
APO_ADDRESS                         61677 non-null object
OWNERSHIP_TYPE                      61677 non-null object
COUNTY_LANDUSE_DESCRIPTION          61677 non-null object
TRPA_LANDUSE_DESCRIPTION            61677 non-null object
TAZ                                 61534 non-null float64
ZONING_ID                           61677 non-null object
ZONING_DESCRIPTION                  61677 non-null object
TOWN_CENTER                         61677 non-null object
LOCATION_TO_TOWNCENTER              61677 non-null object
ALLOWABLE_COVERAGE_BAILEY_SQFT      61670 non-null float64
IMPERVIOUS_SURFACE_SQFT             61677 non-null float64
UNITS                               61677 non-null object
BEDROOMS                            61677 non-null object
BATHROOMS         

In [81]:
dfFC = df[['APN',                                 
            'JURISDICTION',                        
            'APO_ADDRESS',                         
            'OWNERSHIP_TYPE',                      
            'COUNTY_LANDUSE_DESCRIPTION',          
            'TRPA_LANDUSE_DESCRIPTION',
            'TAZ',
            'ZONING_ID',                           
            'ZONING_DESCRIPTION',                  
            'TOWN_CENTER',                         
            'LOCATION_TO_TOWNCENTER',
            'HRA_NAME',                            
            'WATERSHED_NAME',                      
            'FIREPD',                              
            'WITHIN_TRPA_BNDY',                    
            'LITTORAL', 
            'ALLOWABLE_COVERAGE_BAILEY_SQFT',      
            'IMPERVIOUS_SURFACE_SQFT',
            'Potential_Coverage',
            'UNITS',                               
            'BEDROOMS',                            
            'BATHROOMS',                                                      
            'AS_LANDVALUE',                        
            'AS_IMPROVALUE',                       
            'AS_SUM',                              
            'TAX_LANDVALUE',                       
            'TAX_IMPROVALUE',                      
            'TAX_SUM',                             
            'TAX_YEAR',                                                   
            'Vacant_Buildable',                    
            'MultiFamily_Allowed',                 
            'MultiFamily_UnitsPerAcre_Allowed',    
            'SingleFamily_Allowed',
            'Existing_Residential_Units',   
            'DwellingUnits_PerAcre',
            'Max_Residential_Units',               
            'Potential_Residential_Units', 
            'Commercial_Allowed',
            'Existing_CommercialFloorArea',
            'Tourist_Allowed',                     
            'Existing_Tourist_Units',              
            'RBU',                                 
            'RFA',                                 
            'SFRUU',                               
            'CFA',                                 
            'TAU',                                 
            'TFA',                                 
            'BMPStatus',                           
            'RetiredFromDevelopment',                                     
            'Developable_Area',                        
            'EstimatedOrVerified',                 
            'BaseAllowableCoveragePercent',        
            'IPESScore',                           
            'IPESScoreType',                       
            'IPESTotalAllowableCoverageSqFt',                                                     
            'DevelopmentRight',                    
            'LandCapability',                      
            'BankedQuanitity',                     
            'CurrentInventory',                    
            'DateBankedOrApproved',                
            'LastUpdated',                        
            'PARCEL_SQFT',                         
            'PARCEL_ACRES',
            'SHAPE']]

In [82]:
dfFC

,APN,JURISDICTION,APO_ADDRESS,OWNERSHIP_TYPE,COUNTY_LANDUSE_DESCRIPTION,TRPA_LANDUSE_DESCRIPTION,TAZ,ZONING_ID,ZONING_DESCRIPTION,TOWN_CENTER,LOCATION_TO_TOWNCENTER,HRA_NAME,WATERSHED_NAME,FIREPD,WITHIN_TRPA_BNDY,LITTORAL,ALLOWABLE_COVERAGE_BAILEY_SQFT,IMPERVIOUS_SURFACE_SQFT,Potential_Coverage,UNITS,BEDROOMS,BATHROOMS,AS_LANDVALUE,AS_IMPROVALUE,AS_SUM,TAX_LANDVALUE,TAX_IMPROVALUE,TAX_SUM,TAX_YEAR,Vacant_Buildable,MultiFamily_Allowed,MultiFamily_UnitsPerAcre_Allowed,SingleFamily_Allowed,Existing_Residential_Units,DwellingUnits_PerAcre,Max_Residential_Units,Potential_Residential_Units,Commercial_Allowed,Existing_CommercialFloorArea,Tourist_Allowed,Existing_Tourist_Units,RBU,RFA,SFRUU,CFA,TAU,TFA,BMPStatus,RetiredFromDevelopment,Developable_Area,EstimatedOrVerified,BaseAllowableCoveragePercent,IPESScore,IPESScoreType,IPESTotalAllowableCoverageSqFt,DevelopmentRight,LandCapability,BankedQuanitity,CurrentInventory,DateBankedOrApproved,LastUpdated,PARCEL_SQFT,PARCEL_ACRES,SHAPE
0,048-042-03,WA,0 MOUNT ROSE HWY,Federal,"Vacant, single family",Vacant,269.00,053,INCLINE LAKE,,Outside Buffer,Incline,THIRD CREEK,,1,0,"47,242.03",0.00,"47,242.03",0,0,0,261641,0,261641,747546,0,747546,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,"4,693,108.00",Verified,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"4,745,879.49",108.95,"{'rings': [[[765625.7054114696, 4354631.387734..."
1,048-140-01,WA,0 MOUNT ROSE HWY,Federal,"Vacant, single family",Vacant,nan,,,,,,,,0,0,0.00,0.00,0.00,0,0,0,14000,0,14000,40000,0,40000,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"78,267.47",1.80,"{'rings': [[[765719.1469960865, 4354915.431587..."
2,048-140-02,WA,0 MOUNT ROSE HWY,Federal,"Splinter, unbuildable: small size or shape",Vacant,nan,,,,,,,,0,0,0.00,0.00,0.00,0,0,0,28,0,28,80,0,80,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"3,338.13",0.08,"{'rings': [[[765649.5035075517, 4354813.753804..."
3,048-140-03,WA,0 MOUNT ROSE HWY,Local,"Vacant, single family",Vacant,285.00,053,INCLINE LAKE,,Outside Buffer,Incline,THIRD CREEK,,1,0,"2,141.35",0.00,"2,141.35",0,0,0,602455,0,602455,1721300,0,1721300,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,"205,797.00",Verified,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"217,843.29",5.00,"{'rings': [[[765528.4230274847, 4354647.265832..."
4,048-140-04,WA,0 MOUNT ROSE HWY,Federal,"Vacant, other or unknown",Vacant,286.00,053,INCLINE LAKE,,Outside Buffer,Incline,THIRD CREEK,,1,0,"250,371.76","17,877.22","232,494.53",0,0,0,1559172,0,1559172,4454778,0,4454778,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,"25,290,253.00",Verified,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"28,126,585.80",645.70,"{'rings': [[[765065.9830036154, 4355188.277743..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61672,037-020-014,EL,0,Federal,VAC RES RURAL RES. LAND 20+ MINOR NON-RES IMPR,Vacant,nan,,,,,,,LAKE VALLEY FPD,0,0,"2,886.07",0.00,"2,886.07",0,0,,0,0,0,0,0,0,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"7,319,504.76",168.03,"{'rings': [[[755981.943499105, 4297124.9795018..."
61673,038-010-015,EL,0,Federal,VAC RES RURAL RES. LAND 20+ MINOR NON-RES IMPR,Vacant,nan,,,,,,,,0,0,54.77,0.00,54.77,0,0,,0,0,0,0,0,0,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"7,030,322.6

In [83]:
dfFC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61677 entries, 0 to 61676
Data columns (total 64 columns):
APN                                 61677 non-null object
JURISDICTION                        61677 non-null object
APO_ADDRESS                         61677 non-null object
OWNERSHIP_TYPE                      61677 non-null object
COUNTY_LANDUSE_DESCRIPTION          61677 non-null object
TRPA_LANDUSE_DESCRIPTION            61677 non-null object
TAZ                                 61534 non-null float64
ZONING_ID                           61677 non-null object
ZONING_DESCRIPTION                  61677 non-null object
TOWN_CENTER                         61677 non-null object
LOCATION_TO_TOWNCENTER              61677 non-null object
HRA_NAME                            61677 non-null object
WATERSHED_NAME                      61677 non-null object
FIREPD                              61677 non-null object
WITHIN_TRPA_BNDY                    61677 non-null int64
LITTORAL             

In [84]:
df = dfFC
df.spatial.to_featureclass(location= "C:\GIS\PROJECT\DevelopmentPotential\DevelopmentPotential.gdb\ParcelDevelopment")

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\geo\_accessor.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._data[col] = GeoArray(self._data[col])


'C:\\GIS\\PROJECT\\DevelopmentPotential\\DevelopmentPotential.gdb\\ParcelDevelopment'

## Pickle the Data Frame

In [85]:
# dump pickle of merged dataframe
with open(r"C:\GIS\PROJECT\DevelopmentPotential\Data\Pickles\sdfParcelDevelopment.pickle", 'wb') as f:
    pickle.dump(df, f)

## Un-Pickle to Data Frame

In [86]:
# load pickle of merged dataframe
with open(r"C:\GIS\PROJECT\DevelopmentPotential\Data\Pickles\sdfParcelDevelopment.pickle", 'rb') as f:
    df = pickle.load(f)
df

,APN,JURISDICTION,APO_ADDRESS,OWNERSHIP_TYPE,COUNTY_LANDUSE_DESCRIPTION,TRPA_LANDUSE_DESCRIPTION,TAZ,ZONING_ID,ZONING_DESCRIPTION,TOWN_CENTER,LOCATION_TO_TOWNCENTER,HRA_NAME,WATERSHED_NAME,FIREPD,WITHIN_TRPA_BNDY,LITTORAL,ALLOWABLE_COVERAGE_BAILEY_SQFT,IMPERVIOUS_SURFACE_SQFT,Potential_Coverage,UNITS,BEDROOMS,BATHROOMS,AS_LANDVALUE,AS_IMPROVALUE,AS_SUM,TAX_LANDVALUE,TAX_IMPROVALUE,TAX_SUM,TAX_YEAR,Vacant_Buildable,MultiFamily_Allowed,MultiFamily_UnitsPerAcre_Allowed,SingleFamily_Allowed,Existing_Residential_Units,DwellingUnits_PerAcre,Max_Residential_Units,Potential_Residential_Units,Commercial_Allowed,Existing_CommercialFloorArea,Tourist_Allowed,Existing_Tourist_Units,RBU,RFA,SFRUU,CFA,TAU,TFA,BMPStatus,RetiredFromDevelopment,Developable_Area,EstimatedOrVerified,BaseAllowableCoveragePercent,IPESScore,IPESScoreType,IPESTotalAllowableCoverageSqFt,DevelopmentRight,LandCapability,BankedQuanitity,CurrentInventory,DateBankedOrApproved,LastUpdated,PARCEL_SQFT,PARCEL_ACRES,SHAPE
0,048-042-03,WA,0 MOUNT ROSE HWY,Federal,"Vacant, single family",Vacant,269.00,053,INCLINE LAKE,,Outside Buffer,Incline,THIRD CREEK,,1,0,"47,242.03",0.00,"47,242.03",0,0,0,261641,0,261641,747546,0,747546,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,"4,693,108.00",Verified,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"4,745,879.49",108.95,"{'rings': [[[765625.7054114696, 4354631.387734..."
1,048-140-01,WA,0 MOUNT ROSE HWY,Federal,"Vacant, single family",Vacant,nan,,,,,,,,0,0,0.00,0.00,0.00,0,0,0,14000,0,14000,40000,0,40000,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"78,267.47",1.80,"{'rings': [[[765719.1469960865, 4354915.431587..."
2,048-140-02,WA,0 MOUNT ROSE HWY,Federal,"Splinter, unbuildable: small size or shape",Vacant,nan,,,,,,,,0,0,0.00,0.00,0.00,0,0,0,28,0,28,80,0,80,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"3,338.13",0.08,"{'rings': [[[765649.5035075517, 4354813.753804..."
3,048-140-03,WA,0 MOUNT ROSE HWY,Local,"Vacant, single family",Vacant,285.00,053,INCLINE LAKE,,Outside Buffer,Incline,THIRD CREEK,,1,0,"2,141.35",0.00,"2,141.35",0,0,0,602455,0,602455,1721300,0,1721300,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,"205,797.00",Verified,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"217,843.29",5.00,"{'rings': [[[765528.4230274847, 4354647.265832..."
4,048-140-04,WA,0 MOUNT ROSE HWY,Federal,"Vacant, other or unknown",Vacant,286.00,053,INCLINE LAKE,,Outside Buffer,Incline,THIRD CREEK,,1,0,"250,371.76","17,877.22","232,494.53",0,0,0,1559172,0,1559172,4454778,0,4454778,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,"25,290,253.00",Verified,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"28,126,585.80",645.70,"{'rings': [[[765065.9830036154, 4355188.277743..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61672,037-020-014,EL,0,Federal,VAC RES RURAL RES. LAND 20+ MINOR NON-RES IMPR,Vacant,nan,,,,,,,LAKE VALLEY FPD,0,0,"2,886.07",0.00,"2,886.07",0,0,,0,0,0,0,0,0,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"7,319,504.76",168.03,"{'rings': [[[755981.943499105, 4297124.9795018..."
61673,038-010-015,EL,0,Federal,VAC RES RURAL RES. LAND 20+ MINOR NON-RES IMPR,Vacant,nan,,,,,,,,0,0,54.77,0.00,54.77,0,0,,0,0,0,0,0,0,2019,NaN,None,nan,None,nan,nan,nan,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,"7,030,322.6

# Analysis

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61677 entries, 0 to 61676
Data columns (total 64 columns):
APN                                 61677 non-null object
JURISDICTION                        61677 non-null object
APO_ADDRESS                         61677 non-null object
OWNERSHIP_TYPE                      61677 non-null object
COUNTY_LANDUSE_DESCRIPTION          61677 non-null object
TRPA_LANDUSE_DESCRIPTION            61677 non-null object
TAZ                                 61534 non-null float64
ZONING_ID                           61677 non-null object
ZONING_DESCRIPTION                  61677 non-null object
TOWN_CENTER                         61677 non-null object
LOCATION_TO_TOWNCENTER              61677 non-null object
HRA_NAME                            61677 non-null object
WATERSHED_NAME                      61677 non-null object
FIREPD                              61677 non-null object
WITHIN_TRPA_BNDY                    61677 non-null int64
LITTORAL             

## Town Centers

In [88]:
# town center land use, summarized acres
TC_LandUse_Pivot = pd.pivot_table(df,index=['TOWN_CENTER', 'TRPA_LANDUSE_DESCRIPTION'],values=["PARCEL_ACRES"],aggfunc=np.sum)
TC_LandUse_Pivot

PARCEL_ACRES
TOWN_CENTER                        TRPA_LANDUSE_DESCRIPTION               
                                                                  4,437.13
                                   Commercial                     3,605.48
                                   Condominium                      157.13
                                   Condominium Common Area        1,660.58
                                   Multi-Family Residential       1,362.13
                                   Open Space                    96,779.00
                                   Public Service                 9,376.59
                                   Recreation                     8,441.29
                                   Single Family Residential     12,724.20
                                   Tourist Accommodation            116.79
                                   Vacant                       101,907.10
BIJOU/AL TAHOE                                                        0.23
                                   Commercial                        60.13
                                   Multi-Family Residential          11.97
                                   Open Space                         1.19
                                   Public Service                     0.95
                                   Recreation                         1.77
                                   Single Family Residential          9.56
                                   Tourist Accommodation             31.29
                                   Vacant                            32.41
INCLINE VILLAGE COMMERCIAL         Commercial                       143.36
                                   Condominium                        2.02
                                   Condominium Common Area            9.79
                                   Multi-Family Residential           0.21
                                   Open Space                         3.97
                                   Public Service                     0.63
                                   Recreation                         3.53
                                   Vacant                             8.64
INCLINE VILLAGE TOURIST            Commercial                        21.73
                                   Condominium                        2.66
                                   Condominium Common Area           12.27
                                   Multi-Family Residential           0.29
                                   Single Family Residential          0.23
                                   Tourist Accommodation             16.45
                                   Vacant                             0.34
KINGS BEACH COMMERCIAL             Commercial                        22.17
                                   Condominium                        2.52
                                   Condominium Common Area            9.53
                                   Multi-Family Residential           9.83
                                   Open Space                         0.84
                                   Public Service                    20.82
                                   Recreation                        11.89
                                   Single Family Residential          7.85
                                   Tourist Accommodation              8.84
                                   Vacant                            14.52
KINGSBURY COMMERCIAL               Commercial                        64.13
                                   Multi-Family Residential           8.18
                                   Open Space                         0.27
                                   Public Service                     0.67
                                   Single Family Residential          1.33
                                   Tourist Accommodation              6.78
                                   Vacant                             7.43
MEYERS COMME

In [89]:
# Town Center / MF Allowed / Density Allowed / Exising land Use, summarized acres
TC_LandUse_Pivot = pd.pivot_table(df,index=['TOWN_CENTER', 'MultiFamily_Allowed', 'MultiFamily_UnitsPerAcre_Allowed', 'TRPA_LANDUSE_DESCRIPTION'],values=["PARCEL_ACRES"],aggfunc=np.sum)
TC_LandUse_Pivot

PARCEL_ACRES
TOWN_CENTER                        MultiFamily_Allowed MultiFamily_UnitsPerAcre_Allowed TRPA_LANDUSE_DESCRIPTION               
                                   Yes                 3.00                             Condominium Common Area           15.47
                                                                                        Single Family Residential          2.24
                                                       4.00                             Commercial                         1.40
                                                                                        Condominium                        0.11
                                                                                        Condominium Common Area            0.61
                                                                                        Vacant                             0.13
                                                       8.00                                                                1.97
                                                                                        Commercial                        91.72
                                                                                        Condominium                        9.53
                                                                                        Condominium Common Area          132.14
                                                                                        Multi-Family Residential          46.25
                                                                                        Open Space                       246.59
                                                                                        Public Service                    46.79
                                                                                        Recreation                         1.71
                                                                                        Single Family Residential        383.87
                                                                                        Tourist Accommodation             17.72
                                                                                        Vacant                           131.87
                                                       10.00                            Condominium Common Area            5.56
                                                                                        Multi-Family Residential           1.19
                                                                                        Open Space                        28.75
                                                                                        Public Service                     1.61
                                                                                        Single Family Residential         66.36
                                                                                        Vacant                             0.56
                                                       11.00                            Multi-Family Residential           3.21
                                                       12.00                                                               0.14
                                                                                        Commercial                         0.65
                                                                                        Multi-Family Residential          27.93
                                                                                        Open Space                         3.14
                                                                                        Public Service                     2.24
                                                                                        Recreation                         3.58
                   

## TAZ 

In [93]:
# TAZ / Exising land Use, summarized acres
TAZ_LandUse_Pivot = pd.pivot_table(df,index=['TAZ', 'Vacant_Buildable'],values=["PARCEL_ACRES"],aggfunc='count')
TAZ_LandUse_Pivot

PARCEL_ACRES
TAZ    Vacant_Buildable              
9.00   Maybe                        4
11.00  Maybe                       19
       Yes                          3
12.00  Maybe                       42
       Yes                          4
13.00  Maybe                       30
       Yes                          3
14.00  Maybe                       31
15.00  Maybe                       29
       Yes                          2
19.00  Yes                          1
20.00  Maybe                        1
21.00  Maybe                       28
22.00  Maybe                        2
23.00  Maybe                       41
24.00  Maybe                       16
       Yes                          1
25.00  Maybe                        8
       Yes                          1
26.00  Maybe                       19
34.00  Maybe                       26
       Yes                          1
35.00  Maybe                       30
37.00  Maybe                       18
       Yes                          2
38.00  Maybe                       62
       Yes                          1
39.00  Maybe                       47
       Yes                          3
40.00  Maybe                        9
       Yes                          1
41.00  Maybe                       21
       Yes                          3
42.00  Maybe                       91
       Yes                          9
43.00  Maybe                       51
       Yes                          1
45.00  Maybe                       13
       Yes                          4
46.00  Maybe                       22
       Yes                          3
47.00  Maybe                       45
       Yes                          1
48.00  Maybe                       25
49.00  Maybe                        8
51.00  Maybe                       15
       Yes                          1
52.00  Maybe                       10
       Yes                          3
53.00  Maybe                        3
54.00  Maybe                       76
       Yes                          2
56.00  Maybe                       24
57.00  Yes                          1
58.00  Maybe                        3
59.00  Maybe                       66
       Yes                          2
60.00  Maybe                        3
62.00  Maybe                       19
       Yes                          4
64.00  Maybe                        1
65.00  Maybe                       67
       Yes                          2
66.00  Maybe                       27
67.00  Maybe                       31
68.00  Maybe                       23
69.00  Maybe                        9
70.00  Maybe                       25
71.00  Maybe                       11
75.00  Maybe                      173
       Yes                          1
76.00  Maybe                       61
77.00  Maybe                        8
78.00  Maybe                       64
79.00  Maybe                       73
82.00  Maybe                        6
83.00  Maybe                        3
       Yes                          1
85.00  Maybe                       17
       Yes                          1
86.00  Maybe                       28
       Yes                         10
87.00  Yes                          1
88.00  Maybe                       11
       Yes                          1
91.00  Maybe                       53
92.00  Maybe                       51
       Yes                          2
93.00  Maybe                        1
94.00  Maybe                        2
95.00  Maybe                      115
       Yes                          1
96.00  Maybe                        6
97.00  Maybe                       74
       Yes                          1
100.00 Maybe                       76
       Yes                          1
102.00 Maybe                       16
108.00 Maybe                        5
109.00 Maybe                        1
110.00 Maybe                        1
114.00 Maybe                        2
       Yes                          1
115.00 Maybe                       

In [100]:
# TAZ / Exising land Use, summarized acres
TAZ_LandUse_Pivot = pd.pivot_table(df,index=['TAZ', 'TRPA_LANDUSE_DESCRIPTION'],values=["PARCEL_ACRES"],aggfunc=np.sum)
TAZ_LandUse_Pivot

PARCEL_ACRES
TAZ    TRPA_LANDUSE_DESCRIPTION               
9.00                                      0.91
       Commercial                        17.87
       Multi-Family Residential           3.28
       Open Space                         0.28
       Public Service                     2.21
...                                        ...
297.00 Commercial                         0.54
       Condominium                        0.18
       Single Family Residential         14.98
       Tourist Accommodation             16.31
       Vacant                             1.67

[1389 rows x 1 columns]

In [103]:
pd.pivot_table(df,index=["TAZ", 'Vacant_Buildable'],values=["APN"],columns=["MultiFamily_Allowed",'SingleFamily_Allowed'],aggfunc=pd.Series.nunique)

APN
MultiFamily_Allowed      Yes
SingleFamily_Allowed     Yes
TAZ    Vacant_Buildable     
9.00   Maybe               4
11.00  Maybe               1
12.00  Maybe              40
       Yes                 4
13.00  Maybe              29
       Yes                 3
15.00  Maybe               2
19.00  Yes                 1
21.00  Maybe               3
23.00  Maybe               6
24.00  Maybe              16
       Yes                 1
25.00  Maybe               8
       Yes                 1
26.00  Maybe               4
35.00  Maybe              14
37.00  Maybe              18
       Yes                 2
38.00  Maybe              51
40.00  Maybe               6
       Yes                 1
41.00  Maybe               3
       Yes                 1
42.00  Maybe              16
       Yes                 6
43.00  Maybe              10
46.00  Maybe               8
       Yes                 2
48.00  Maybe               2
49.00  Maybe               6
53.00  Maybe               2
54.00  Maybe               3
57.00  Yes                 1
59.00  Maybe              13
       Yes                 2
60.00  Maybe               3
62.00  Maybe               8
       Yes                 2
85.00  Maybe               2
86.00  Maybe               2
91.00  Maybe               7
93.00  Maybe               1
96.00  Maybe               2
129.00 Maybe              20
130.00 Maybe             176
       Yes                 3
131.00 Maybe               8
133.00 Maybe               1
       Yes                 1
134.00 Maybe               9
       Yes                 1
139.00 Maybe               1
153.00 Yes                 1
157.00 Yes                 2
160.00 Maybe               3
       Yes                 1
166.00 Maybe               3
       Yes                 3
170.00 Maybe               2
171.00 Maybe               4
181.00 Maybe               9
       Yes                 1
190.00 Maybe              48
       Yes                 5
191.00 Yes                 1
205.00 Yes                 1
206.00 Maybe               4
213.00 Yes                 1
221.00 Maybe               8
223.00 Maybe               2
240.00 Maybe               9
       Yes                 1
242.00 Maybe               7
       Yes                 1
267.00 Maybe              14
274.00 Maybe               1
279.00 Maybe               8
282.00 Maybe               1
287.00 Maybe               1
291.00 Maybe              13

## Vacant Parcels

In [107]:
vacant = df[((df.TRPA_LANDUSE_DESCRIPTION == "Vacant")
            & (df.OWNERSHIP_TYPE == "Private")
            & (df.RetiredFromDevelopment == 'No')
            & ((df.JURISDICTION.isin(['EL','CSLT','DG','WA','CC'])) & (df.IPESScore > 0))
            | (df.TRPA_LANDUSE_DESCRIPTION == "Vacant")
            & (df.OWNERSHIP_TYPE == "Private") 
            & (df.RetiredFromDevelopment == 'No')
            & ((df.JURISDICTION == "PL") & (df.IPESScore>726)))]
vacant

,APN,JURISDICTION,APO_ADDRESS,OWNERSHIP_TYPE,COUNTY_LANDUSE_DESCRIPTION,TRPA_LANDUSE_DESCRIPTION,TAZ,ZONING_ID,ZONING_DESCRIPTION,TOWN_CENTER,LOCATION_TO_TOWNCENTER,HRA_NAME,WATERSHED_NAME,FIREPD,WITHIN_TRPA_BNDY,LITTORAL,ALLOWABLE_COVERAGE_BAILEY_SQFT,IMPERVIOUS_SURFACE_SQFT,Potential_Coverage,UNITS,BEDROOMS,BATHROOMS,AS_LANDVALUE,AS_IMPROVALUE,AS_SUM,TAX_LANDVALUE,TAX_IMPROVALUE,TAX_SUM,TAX_YEAR,Vacant_Buildable,MultiFamily_Allowed,MultiFamily_UnitsPerAcre_Allowed,SingleFamily_Allowed,Existing_Residential_Units,DwellingUnits_PerAcre,Max_Residential_Units,Potential_Residential_Units,Commercial_Allowed,Existing_CommercialFloorArea,Tourist_Allowed,Existing_Tourist_Units,RBU,RFA,SFRUU,CFA,TAU,TFA,BMPStatus,RetiredFromDevelopment,Developable_Area,EstimatedOrVerified,BaseAllowableCoveragePercent,IPESScore,IPESScoreType,IPESTotalAllowableCoverageSqFt,DevelopmentRight,LandCapability,BankedQuanitity,CurrentInventory,DateBankedOrApproved,LastUpdated,PARCEL_SQFT,PARCEL_ACRES,SHAPE
143,122-100-14,WA,0 LAKESHORE BLVD,Private,"Vacant, single family",Vacant,288.00,037,LAKEVIEW,,Outside Buffer,Incline,SECOND CREEK,NORTH LAKE TAHOE FPD,1,1,"8,494.01","3,365.78","5,128.22",0,0,0,475566,750,476316,1358760,2142,1360902,2019,Yes,None,nan,Yes,nan,nan,1.00,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,25%,804.00,Official,nan,NaN,NaN,nan,nan,NaN,NaN,"44,680.87",1.03,"{'rings': [[[760580.4229420656, 4348749.725502..."
486,122-181-44,WA,813 LAKESHORE BLVD,Private,"Vacant, single family",Vacant,275.00,037,LAKEVIEW,,Quarter Mile Buffer,Incline,BURNT CEDAR CREEK,NORTH LAKE TAHOE FPD,1,1,"5,400.53","1,261.07","4,139.46",0,0,0,634088,582,634670,1811680,1664,1813344,2019,Yes,None,nan,Yes,nan,nan,1.00,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,26%,872.00,Official,nan,NaN,NaN,nan,nan,NaN,NaN,"20,824.43",0.48,"{'rings': [[[762167.0977808535, 4348304.464276..."
497,122-181-64,WA,885 LAKESHORE BLVD,Private,"Vacant, single family",Vacant,275.00,037,LAKEVIEW,,Outside Buffer,Incline,WOOD CREEK,NORTH LAKE TAHOE FPD,1,1,"9,312.12","1,835.11","7,477.01",0,0,0,1664481,4998,1669479,4755660,14280,4769940,2019,Yes,None,nan,Yes,1.00,1.13,1.00,0.00,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Certified,No,nan,NaN,26%,891.00,Official,nan,NaN,NaN,nan,nan,NaN,NaN,"38,554.06",0.89,"{'rings': [[[762753.0198843945, 4348116.611707..."
595,122-194-19,WA,720 MAYS BLVD,Private,"Vacant, single family",Vacant,277.00,037,LAKEVIEW,,Quarter Mile Buffer,Incline,BURNT CEDAR CREEK,NORTH LAKE TAHOE FPD,1,0,"5,574.10",12.77,"5,561.33",0,0,0,252000,0,252000,720000,0,720000,2019,Maybe,None,nan,Yes,nan,nan,1.00,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,28%,859.00,Official,nan,NaN,NaN,nan,nan,NaN,NaN,"18,580.33",0.43,"{'rings': [[[761756.0123485299, 4348734.277805..."
643,122-201-21,WA,740 LAKESHORE BLVD,Private,"Vacant, single family",Vacant,277.00,037,LAKEVIEW,,Quarter Mile Buffer,Incline,BURNT CEDAR CREEK,NORTH LAKE TAHOE FPD,1,0,"6,001.53",0.00,"6,001.53",0,0,0,455000,0,455000,1300000,0,1300000,2019,Maybe,None,nan,Yes,nan,nan,1.00,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,29%,939.00,Official,"5,806.00",NaN,NaN,nan,nan,NaN,NaN,"20,005.10",0.46,"{'rings': [[[761821.1902377997, 4348500.300644..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60605,029-402-034,CSLT,3841 FIGUEROA LN,Private,VAC RES VACANT RES. LAND <=2.5 AC. 1-3 UNITS,Vacant,12.00,092,PIONEER/SKI RUN,,Quarter Mile Buffer,South Stateline,BIJOU PARK,CSLT FPD,1,0,"1,257.17",0.00,"1,257.17",0,0,,40800,0,40800,40800,0,40800,2019,Maybe,Yes,15.00,Yes,nan,nan,2.00,nan,Yes,nan,Yes,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,26%,752.00,Official,"1,300.00",NaN,NaN,nan,nan,NaN,NaN,"

In [112]:
vacant = df[((df.OWNERSHIP_TYPE == "Private")
            & (df.IMPERVIOUS_SURFACE_SQFT < 1000)
            & (df.RetiredFromDevelopment == 'No')
            & ((df.JURISDICTION.isin(['EL','CSLT','DG','WA','CC'])) & (df.IPESScore > 0))
            | (df.OWNERSHIP_TYPE == "Private") 
            & (df.RetiredFromDevelopment == 'No')
            & (df.IMPERVIOUS_SURFACE_SQFT < 1000)
            & ((df.JURISDICTION == "PL") & (df.IPESScore>726)))]

vacant

,APN,JURISDICTION,APO_ADDRESS,OWNERSHIP_TYPE,COUNTY_LANDUSE_DESCRIPTION,TRPA_LANDUSE_DESCRIPTION,TAZ,ZONING_ID,ZONING_DESCRIPTION,TOWN_CENTER,LOCATION_TO_TOWNCENTER,HRA_NAME,WATERSHED_NAME,FIREPD,WITHIN_TRPA_BNDY,LITTORAL,ALLOWABLE_COVERAGE_BAILEY_SQFT,IMPERVIOUS_SURFACE_SQFT,Potential_Coverage,UNITS,BEDROOMS,BATHROOMS,AS_LANDVALUE,AS_IMPROVALUE,AS_SUM,TAX_LANDVALUE,TAX_IMPROVALUE,TAX_SUM,TAX_YEAR,Vacant_Buildable,MultiFamily_Allowed,MultiFamily_UnitsPerAcre_Allowed,SingleFamily_Allowed,Existing_Residential_Units,DwellingUnits_PerAcre,Max_Residential_Units,Potential_Residential_Units,Commercial_Allowed,Existing_CommercialFloorArea,Tourist_Allowed,Existing_Tourist_Units,RBU,RFA,SFRUU,CFA,TAU,TFA,BMPStatus,RetiredFromDevelopment,Developable_Area,EstimatedOrVerified,BaseAllowableCoveragePercent,IPESScore,IPESScoreType,IPESTotalAllowableCoverageSqFt,DevelopmentRight,LandCapability,BankedQuanitity,CurrentInventory,DateBankedOrApproved,LastUpdated,PARCEL_SQFT,PARCEL_ACRES,SHAPE
448,122-161-09,WA,759 LAKESHORE BLVD,Private,Single Family Residence,Single Family Residential,277.00,037,LAKEVIEW,,Quarter Mile Buffer,Incline,BURNT CEDAR CREEK,NORTH LAKE TAHOE FPD,1,0,"5,133.95",0.00,"5,133.95",1,4,5.5,386750,566253,953003,1105000,1617866,2722866,2019,Maybe,None,nan,Yes,nan,nan,1.00,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Certified,No,nan,NaN,30%,986.00,Official,"5,130.00",NaN,NaN,nan,nan,NaN,NaN,"17,113.17",0.39,"{'rings': [[[761868.3220300404, 4348401.695160..."
595,122-194-19,WA,720 MAYS BLVD,Private,"Vacant, single family",Vacant,277.00,037,LAKEVIEW,,Quarter Mile Buffer,Incline,BURNT CEDAR CREEK,NORTH LAKE TAHOE FPD,1,0,"5,574.10",12.77,"5,561.33",0,0,0,252000,0,252000,720000,0,720000,2019,Maybe,None,nan,Yes,nan,nan,1.00,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,28%,859.00,Official,nan,NaN,NaN,nan,nan,NaN,NaN,"18,580.33",0.43,"{'rings': [[[761756.0123485299, 4348734.277805..."
609,122-195-06,WA,727 MARTIS PEAK DR,Private,Single Family Residence,Single Family Residential,277.00,037,LAKEVIEW,,Quarter Mile Buffer,Incline,BURNT CEDAR CREEK,NORTH LAKE TAHOE FPD,1,0,"5,436.06",0.00,"5,436.06",1,5,5.5,252000,537232,789232,720000,1534949,2254949,2019,Maybe,None,nan,Yes,nan,nan,1.00,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Certified,No,nan,NaN,29%,954.00,Official,"5,253.00",NaN,NaN,nan,nan,NaN,NaN,"18,120.21",0.42,"{'rings': [[[761796.193641915, 4348893.9087792..."
643,122-201-21,WA,740 LAKESHORE BLVD,Private,"Vacant, single family",Vacant,277.00,037,LAKEVIEW,,Quarter Mile Buffer,Incline,BURNT CEDAR CREEK,NORTH LAKE TAHOE FPD,1,0,"6,001.53",0.00,"6,001.53",0,0,0,455000,0,455000,1300000,0,1300000,2019,Maybe,None,nan,Yes,nan,nan,1.00,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,29%,939.00,Official,"5,806.00",NaN,NaN,nan,nan,NaN,NaN,"20,005.10",0.46,"{'rings': [[[761821.1902377997, 4348500.300644..."
710,122-212-06,WA,835 OPHIR PEAK RD,Private,"Vacant, single family",Vacant,276.00,037,LAKEVIEW,,Quarter Mile Buffer,Incline,WOOD CREEK,NORTH LAKE TAHOE FPD,1,0,"5,941.85",0.00,"5,941.85",0,0,0,252000,0,252000,720000,0,720000,2019,Maybe,None,nan,Yes,nan,nan,1.00,nan,None,nan,None,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,30%,955.00,Official,nan,NaN,NaN,nan,nan,NaN,NaN,"19,806.17",0.45,"{'rings': [[[762485.1262284974, 4348439.146554..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60605,029-402-034,CSLT,3841 FIGUEROA LN,Private,VAC RES VACANT RES. LAND <=2.5 AC. 1-3 UNITS,Vacant,12.00,092,PIONEER/SKI RUN,,Quarter Mile Buffer,South Stateline,BIJOU PARK,CSLT FPD,1,0,"1,257.17",0.00,"1,257.17",0,0,,40800,0,40800,40800,0,40800,2019,Maybe,Yes,15.00,Yes,nan,nan,2.00,nan,Yes,nan,Yes,nan,0.00,0.00,0.00,0.00,0.00,0.00,Not Certified,No,nan,NaN,26%

In [123]:
sum(vacant.groupby('JURISDICTION')['APN'].nunique())

3032

In [124]:
vacant.groupby('JURISDICTION')['APN'].nunique()

JURISDICTION
CSLT     956
DG       115
EL      1338
PL       457
WA       166
Name: APN, dtype: int64

In [118]:
vacantTC = vacant.groupby('TOWN_CENTER')['APN'].nunique()
vacantTC

TOWN_CENTER
                              2993
BIJOU/AL TAHOE                   7
INCLINE VILLAGE COMMERCIAL       7
KINGS BEACH COMMERCIAL           5
MEYERS COMMERCIAL                8
SOUTH Y                         10
STATELINE/SKI RUN                2
Name: APN, dtype: int64

In [117]:
vacantTAZ = vacant.groupby('TAZ')['APN'].nunique()
vacantTAZ


TAZ
9.00        4
11.00      19
12.00      42
13.00      30
14.00      31
15.00      29
20.00       1
21.00      28
22.00       2
23.00      41
24.00      16
25.00       8
26.00      19
34.00      26
35.00      30
37.00      18
38.00      62
39.00      47
40.00       9
41.00      21
42.00      90
43.00      51
45.00      13
46.00      22
47.00      45
48.00      25
49.00       8
51.00      15
52.00      10
53.00       3
54.00      76
56.00      24
58.00       3
59.00      66
60.00       3
62.00      19
64.00       1
65.00      67
66.00      27
67.00      31
68.00      23
69.00       9
70.00      25
71.00      11
75.00     172
76.00      61
77.00       8
78.00      64
79.00      73
82.00       6
83.00       3
85.00      17
86.00      28
88.00      11
91.00      53
92.00      51
93.00       1
94.00       2
95.00     115
96.00       6
97.00      74
100.00     75
102.00     16
108.00      5
109.00      1
110.00      1
114.00      2
115.00     16
121.00      1
122.00     30
124.00     35
12

## Land Use

In [95]:
uses = pd.crosstab(df.COUNTY_LANDUSE_DESCRIPTION, df.TRPA_LANDUSE_DESCRIPTION)
uses

TRPA_LANDUSE_DESCRIPTION,,Commercial,Condominium,Condominium Common Area,Multi-Family Residential,Open Space,Public Service,Recreation,Single Family Residential,Tourist Accommodation,Vacant
COUNTY_LANDUSE_DESCRIPTION,,,,,,,,,,,
,460,0,0,0,0,253,117,0,0,0,0
"2 SINGLE FAM RES, DUPLEX",0,0,0,0,326,0,0,0,0,0,0
"3 SINGLE FAM RES, TRIPLEX",0,0,0,0,42,0,0,0,0,0,0
AGP MSC RURAL RESTRICTIVE ZONING - CLCA (ACTIVE),0,0,0,0,0,7,0,0,0,0,0
AGP MSC RURAL RESTRICTIVE ZONING - NON-RENEWAL,0,0,0,0,0,3,0,0,0,0,0
"APARTMENTS, 4 UNITS OR MORE",0,0,0,0,86,0,0,0,0,0,0
"AUTO SALES, REPAIR",0,8,0,0,0,0,0,0,0,0,0
"BANKS, S&L'S, CREDIT UNION",0,5,0,0,0,0,0,0,0,0,0
"CAMPS & PARKS, GENERAL",0,0,0,0,0,0,0,26,0,0,0
